# Lignes de champ électrostatique


Dans cette partie, on va se poser le problème suivant :

**Comment déterminer la carte de champ électrique créée par une distribution de charges ?**

La résolution se fera en 2 étapes :
* Calcul du potentiel électrique en tout point de l'espace.
* Utilisation de la relation $\vec{E}=- \overrightarrow{\mbox{grad}}(V)$ pour en déduire le champ électrique.

## Un peu de théorie

On se rappelle que le potentiel électrostatique créé par une charge ponctuelle $q$ en un point $M$ situé à une distance $r$ de la charge est :
$$V_q(M)= \frac{q}{4\pi\varepsilon_0 r}$$
où $\varepsilon_0$ est la permittivité diélectrique du vide.

Si $N$ charges $q_i$ sont placées en différents points $O_i$ de l'espace, alors on peut calculer le potentiel en un point $M$ en appliquant le théorème de superposition :
$$V(M) = \sum_{i=1}^N \frac{q_i}{4\pi \varepsilon_0 r_i}$$
où $r_i=O_iM$ est la distance entre le point $M$ et la position de la charge $q_i$.

## Calcul du potentiel électrostatique

Travaillons en deux dimensions dans un domaine $\mathcal{D}$ du plan de $100\mu m \times 100\mu m$. En tout point de ce domaine, le potentiel est caractérisé par un tableau à 2 dimensions ```V``` 100 lignes et 100 colonnes. Ainsi : ```V[i,j]``` sera le potentiel du point de la ```i```ème ligne et ```j```ème colonne et ```V[i,j+1]``` celui d'un point situé $1\mu m$ à sa droite.

On peut donc créer initialement le tableau ```V``` rempli de 0 pour le moment :

In [23]:
import numpy as np
N=100
V=np.zeros((N,N))

Dans notre implémentation, une charge sera représentée par une liste de 3 éléments : ```charge=[valeur_charge, ligne, colonne]```. Le premier élément donnant la valeur de la charge en Coulombs et les deux suivants indiquant l'indice de la ligne et de la colonne où elle est placée.

Puis la distribution de charges sera une liste des charges précédentes : ```distrib=[charge1,charge2...]```.

On donne donc ci-dessous un exemple de distribution sur lequel nous allons travailler par la suite :

In [28]:
e=1.6e-19
distrib=[[-e,40,40],[2*e,60,60],[-e,60,30]]

> **Question 1**
>
> Ecrire une fonction ```visu(distrib)``` permettant de visualiser le domaine $\mathcal{D}$ de l'espace et le placement des charges sous la forme d'une image (les charges peuvent apparaître comme un point coloré sur un fond uniforme). On pourra utiliser la fonction ```imshow``` du module ```matplotlib.pyplot```.

In [29]:
import matplotlib.pyplot as plt

plt.clf()
def visu(distrib):
    tableau=np.copy(V)
    for charge in distrib :
        i,j=charge[1],charge[2]
        tableau[i,j]=1 # la présence d'une charge est matérialisée par un 1 ici
    plt.imshow(tableau)
    
visu(distrib)
plt.show()

Pour tout point $M$ du domaine $\mathcal{D}$, il est important de pouvoir calculer la distance entre le point $M$ et une certaine charge $q_i$ : cela nous permettra d'évaluer la distance $r_i$.

> **Question 2**
>
> Ecrire une fonction ```distance(charge,i,j)``` prenant pour argument une liste ```charge``` et renvoyant sa distance à la case ```[i,j]```. La tester avec ```charge=[1,2,7]```, ```i=7```, ```j=7```. On doit trouver 5$\mu m$.

In [30]:
from math import sqrt

def distance(charge,i,j):
    icharge=charge[1]
    jcharge=charge[2]
    d_pix=sqrt((i-icharge)**2+(j-jcharge)**2)
    ri=d_pix*1e-6
    return ri

distance([1,2,7],7,7)

4.9999999999999996e-06

Nous allons désormais créer une fonction ```potentiel(charge,i,j)``` prenant pour argument une liste ```charge``` et renvoyant le potentiel électrostatique créé par la charge dans la case ```[i,j]```.

> **Question 3**
>
> Ecrire cette fonction.

In [34]:
def potentiel(charge,i,j):
    eps0=8.85e-12
    pi=3.14
    ri=distance(charge,i,j)
    return charge[0]/(4*pi*ri*eps0)

Nous allons bientôt pouvoir calculer le potentiel en tout point du domaine $\mathcal{D}$... enfin pas exactement en tout point : on ne calcule pas le potentiel des points où se situent les charges. Récupérons rapidement donc les coordonnées des points où l'on ne va pas calculer le potentiel.

> **Question 4**
>
> Ecrire la fonction ```position_charges```pour qu'elle retourne la liste des coordonnées des charges de la distribution passée en argument. La tester avec la distribution données plus haut.


In [35]:
def position_charges(distrib):
    coor_charges=[]
    for charge in distrib :
        i,j=charge[1],charge[2]
        coor_charges.append([i,j])
    return coor_charges
    
position_charges(distrib) 

[[40, 40], [60, 60], [60, 30]]

> **Question 5**
>
> Ecrire une fonction ```calcul_potentiel(distrib,V)``` qui calcule en tout point du tableau ```V``` (en dehors des points occupés par les charges) le potentiel créé par la distribution de charges ```distrib```. La fonction retourne le tableau ```V```complété.

> **Question 6**
>
> Afficher ensuite la carte de potentiel ainsi calculée et les équipotentielles à l'aide de la fonction ```contour``` du module ```plot```. On utilisera des échelles de couleur différentes (argument ```cmap```) dans les fonctions ```imshow``` et ```contour```  de manière à faire ressortir les équipotentielles. On consultera à profit l'aide pour l'utilisation de ces fonctions ```help(nom_fonction)```.

In [37]:
#Q5
def calcul_potentiel(distrib,V):
    positions_charges=position_charges(distrib)
    N=100
    for i in range(N):
        for j in range(N):
            if [i,j] not in positions_charges :  # on se place en dehors des points occupés par les charges
                Vij=0
                for charge in distrib :
                    Vij+=potentiel(charge,i,j)
                V[i,j]=Vij

    return V

V=calcul_potentiel(distrib,V)

#Q6
plt.clf()
plt.imshow(V, cmap="winter")
plt.contour(V,200, cmap="Oranges")
plt.show()

## Calcul du champ électrostatique

Maintenant que le potentiel est connu en tout point de l'espace, on va pouvoir calculer le champ électrostatique en appliquant la relation (uniquement valable en électro**statique**) :
$$\vec{E}=-\overrightarrow{\mbox{grad}}(V)$$

Dans le plan, en notant $E_x$ et $E_y$ les composantes du champ électrique, cette relation se projette :
$$E_x=- \frac{\partial V}{\partial x}$$
$$E_y=-\frac{\partial V}{\partial y}$$

Ces relations sont approximables par méthode d'Euler :
$$E_x= - \frac{V[i,j+1]-V[i,j]}{a}$$

$$E_y = - \frac{V[i+1,j]-V[i,j]}{a}$$

où $a=1\mu m$ est la distance entre deux cases adjacentes.

> **Question 7**
>
> Ecrire une fonction ```champE(V,i,j)``` qui détermine le champ électrique au point de coordonnées ```i,j``` du tableau de potentiel V. Cette fonction renvoie deux scalaires : ```Ex,Ey```.

> **Question 8**
>
> Au sein d'une fonction ```champE(V,distrib)```, appliquer la fonction précédente en tout point (hormis ceux de la dernière ligne et de la dernière colonne ; et ceux qui utiliseraient le potentiel à l'endroit d'une charge). La fonction ```champE``` doit renvoyer deux tableaux : celui des composante du champ électrique selon x et selon y, évaluées en tout point du domaine D.

In [38]:
def champE(V,i,j):
    a=1e-6
    Ex=(V[i,j]-V[i,j+1])/a
    Ey=(V[i,j]-V[i+1,j])/a
    return Ex,Ey

In [39]:
def calcul_champE(V,distrib):
    N=100
    Ex=np.zeros((N,N))
    Ey=np.zeros((N,N))
    
    positions_charges=position_charges(distrib)
    
    for i in range(N-1):
        for j in range(N-1):
            if ([i,j] not in positions_charges) and ([i,j+1] not in positions_charges) and ([i+1,j] not in positions_charges):
                Ex[i,j],Ey[i,j]=champE(V,i,j)
    return Ex,Ey

Ex,Ey=calcul_champE(V,distrib)

Le code écrit ci-dessous permet d'afficher la carte de potentiel, les équipotentielles et les lignes de champ du champ électrique. On peut vérifier la cohérence du résultat : le champ électtique diverge bien des charges positives pour converger vers les charges négatives.

In [40]:
import matplotlib
matplotlib.rcParams['figure.figsize']=[9,9]

plt.clf()
plt.imshow(V)
plt.contour(V,200, cmap="Oranges")
plt.streamplot(np.linspace(0,99,100), np.linspace(0,99,100), Ex, Ey, color="white", linewidth=1)  
plt.show()

Toutefois, nous avons peu d'informations sur la norme du champ électrique sur cette représentation. A l'aide de l'argument ```linewidth``` de la fonction ```streamplot``` nous allons faire en sorte que la ligne de champ électrique soit d'autant plus épaisse que la norme du champ électrique est élevée.

On rappelle que ```Ex``` et ```Ey``` sont des tableaux ```numpy```, ce qui simplifie la syntaxe des calculs.

> **Question 9**
>
> Créer le tableau ```normE``` de même dimension que ```Ex``` et ```Ey``` et donnant la norme du champ électrique en tout point. Déterminer ensuite le maximum ```Emax``` de la norme du champ électrique dans le domaine d'étude. On posera ensuite ```lw=normE/Emax``` et on ré-exécutera la fonction ```streamplot``` avec ```linewidth=lw```.

In [41]:
normE=np.sqrt(Ex**2+Ey**2)
Emax=np.max(normE)
lw=normE/Emax

plt.clf()
plt.imshow(V)
plt.contour(V,200, cmap="Oranges")
plt.streamplot(np.linspace(0,99,100), np.linspace(0,99,100), Ex, Ey, color="white", linewidth=75*lw, arrowsize=1) 
plt.show()

**Pour aller plus loin :**

L'image précédente n'est pas très belle au voisinage des charges car le champ électrique y varie rapidement, et on observe alors des discontinuités de la largeur des lignes de champ.

Il faudrait en faite "lisser" les variations rapides au sein du tableau ```lw```. Cette méthode est analogue à certains filtres créant du "flou" en traitement d'image. Ceux-ci étant déjà codés dans ```scipy```, on en profite et, avec un filtre dit "gaussien", on obtient le résultat suivant :

In [42]:
from scipy import misc,ndimage
lw_lisse= ndimage.gaussian_filter(lw, sigma=15)

In [43]:
plt.clf()
plt.imshow(V)
plt.contour(V,200, cmap="Oranges")
plt.streamplot(np.linspace(0,99,100), np.linspace(0,99,100), Ex, Ey, color="white", linewidth=75*lw_lisse, arrowsize=1)
plt.show()

Il reste encore le problème des "effets de bord" du domaine. Il faudrait par exemple que les charges soient plus éloignées des bordures de $\mathcal{D}$. La solution la plus simple dans un premier temps serait d'agrandir le domaine... au prix d'un coût en temps de calcul.